## Streamlit

Welcome to streamlit, a free, open-source framework that allows you to build interactive web apps. We can use it to share our data or machine learning models with a general audience. Our goal is to introduce you to some basic streamlit functionalities, but this is just the tip of the iceberg. Here are some resources if you're excited and want to learn more: 

1. https://docs.streamlit.io/develop/quick-reference/cheat-sheet
2. https://www.youtube.com/watch?v=vIQQR_yq-8I

## Our Goal
We are going to build an app to compare a molecule from the user to existing drug molecules. 
It will have the following functionalities: 
1. A graphical interface to draw a molecule
2. A 3D visualization of that molecule
3. A dashboard displaying the Lipinski descriptors for the molecule (https://en.wikipedia.org/wiki/Lipinski%27s_rule_of_five)
4. A tool to search a database of approved drugs for the most similar molecules
5. A series of plots that compare our molecule's properties to the distribution of drug molecules

## How to use these exercises
Our app will be run from the ```app.py``` file. However, it's harder to troubleshoot from there. We will first develop necessary functions in this jupyter notebook before copying them into the ```app.py``` file. 

### Text 

We assemble our webpage by piecing together individual widgets. Let's start with text. Transfer the following to the ```app.py``` file. 

```
st.title('Are you drug like?!')
st.caption("Practical Proramming In Chemistry Week 11")
st.markdown("Draw a molecule and see how it stacks up against known drugs")
```

In your terminal, type ```streamlit run app.py```. Note that you must be in the folder with the app.py file for this to work. It should launch a browser where you can see your app. 

Hit the three dots in the upper right corner of the app. Go to settings, and check the box that says "Run on Save". This means that every time you save your file, the app will update automatically. 

### Widgets
Streamlit offers a huge variety of widgets for user interaction. One such widget is a slider (but there are many many more, check the cheatsheet)! Add this code to your ```app.py``` file and save. 

```
st.sidebar.markdown('# Options')
slider_val = st.sidebar.slider("Number of similar molecules", 0, 10, 4)
```

When we change the slider in the app, the variable slider_val will change accordingly. 

### Ketcher
The first main feature of the app is a ketcher interface, which allows us to draw a molecule. Luckily, other developers have made this super easy for us! All we need to do is use the ```streamlit_ketcher``` package. 

Let's walk through how. ```st_ketcher()``` creates the ketcher interface. The function can optionally take a smiles and a height. It returns the smiles of the molecule that the user draws, once the user hits the apply button. 

It may be useful to let the user draw a molecule from smiles. To do so, we will use the `st.text_input()` function. We add a default smiles.  

```
with st.expander("Draw Molecule From Smiles (optional)"):
    molecule = st.text_input("**Smiles**", "CCO") # initially, molecule = ""CCO"
```

We create the ketcher as follows: 
```ketcher_smiles = st_ketcher(molecule, height=600) ```

When the user draws a molecule and hits apply, the variable `ketcher_smiles` will be updated accordingly. 

We want to display the `ketcher_smiles` because it might be useful for the user. Write code to display this variable in an expander. 

### 3D visualization
It would be nice to see how our molecule looks in 3D. To do so, we'll make use of two different packages. We can use the RDKit library to generate a 3D structure of our molecule. Then, we can use ```stmol``` to visualize it in streamlit. 

Write a function that takes a smiles and outputs a .mol string of 3D coordinates. Make sure to include error handling, so that the function returns `None` if rdkit cannot generate a valid molecule. Test your function on a few cases, then tranfser it into the `app.py` file. 

In [ ]:
def generate_3D(smiles):
    "Generate 3D coordinates from smiles in a .mol format string"
    pass # your code here!! 
    return mol_string

We've provided you with a function to visualize the molecule in 3D using stmol. We want to put this 3D molecule below our ketcher interface. We've already created two columns using ```st.columns()```. In the first column, put a sensible title and caption. Use your function to generate 3D coordinates for the molecule, and then use the ```visualize_3D()``` function to show the molecule. 
Add error handling using ```st.error()``` to exit gracefully if the ```ketcher_smiles``` is invalid. 

### Descriptors
In the second column, we want to compute descriptors related to Lipinski's rule of 5 (the molecular weight, logP, number of H bond donors, and the number of H bond acceptors) for our ```ketcher_smiles```. Write a function using rdkit that calculates these descriptors. Include appropriate error handling!! 

In [ ]:
def calculate_descriptors(smiles):
    "Calculate Lipinski descriptors: molecular weight, H-bond donors, H-bond acceptors, and LogP"
    pass # your code here!!
    return mw, hbd, hba, logp

It would be nice to know how the Lipinski descriptors for our molecule compare to those of known drugs. To make this comparison, first we need to load our dataset from week 5 of known drugs. Write a function that returns a ```pd.DataFrame``` of the week5 chembl_drugs dataset. Once you're happy with your function, transfer it to the `app.py` file. 

In [ ]:
def download_data():
    "Download the ChEMBL database"
    pass # your code here!!
    return df

We need to calculate the Lipinski descriptors for our imported data. Later, we will want to do a similarity search, so we should also calculate the morgan fingerprints for each molecule. Write a function called ```modify_data(df)``` that takes a dataframe, removes any rows that have `NA` in the "Smiles" column, calculates the four Lipinski descriptors, and calculates the morgan fingerprints. Return the morgan fingerprints seperately, as streamlit can't save the dataframe with fingerprints in it. 

In [ ]:
def modify_data(df):
    "Modify the ChEMBL dataframe to include Lipinski descriptors. Also get the Morgan fingerprints of each molecule."
    return modified_df, fingerprints

Now that we have our functions, let's use them! Add 
```
df = download_data()
df, fingerprints = modify_data(df)
``` 
to the app.py file. 

We will display the Lipinski descriptors using a the ```st.metric()``` widget. This widget is a nice way to depict data. We provide a title, a value, and a delta. Our delta in this case is difference between the descriptors for our molecule and the average for the drugs dataset. We will format all of this in containers, which add pleasant little bounding boxes. We can also add a success widget to show the user whether or not the input molecule passes all of lipinski's rule of 5. 

The outline of this code is already in the app.py file. Finish it for tiles 2-4.


### Caching 
Try changing the molecule in the app. See how long it takes for the app to recalulate descriptors? This is because every time we change the molecule, the entire app runs top to bottom. What this means is our expensive calculations (finding the lipinski descriptors and fingerprints) are rerun every time. This results in a fairly unpleasant user experience. To avoid it, we can use the streamlit decorator `@st.cache_data()`. This decorator makes it so that if the function is rerun with the same inputs, the function doesn't actually run, streamlit just returns the last cached values. In our case, as long as we don't change the input dataframe, our expensive calculations are only run once (at the beginning.) Add the ```@st.cache_data()``` decorator to the line right before the ```download_data()``` and ```modify_data()``` functions. Try changing the molecule now. Big improvement, right??

### Similarity search
Now that we've got our dataset of known drugs, wouldn't it also be nice to see which drug in the database is most similar to our input molecule? Wouldn't it be nice if the user could choose how many hits pop up? 

Write a function that takes all of the fingerprints, the ketcher_smiles, and the slider_val, and returns a dataframe of the top-n most similar molecules. 

In [ ]:
def find_most_similar(fingerprints, ketcher_smiles, slider_val = 4):
    "Find the most similar molecules to the one drawn in Ketcher."
    pass # your code here!!
    return df

We will display the top n hits using the package mols2grid, which allows us to visualize dataframes of molecules. Take a moment to read and understand the following snippet of code, then transfer it to the app.py file. 

```
most_similar_df = find_most_similar(fingerprints, ketcher_smiles, slider_val)
if most_similar_df is not None:
    if slider_val > 8: 
        height = 600 # Dynamically adjust height for visualization
    elif slider_val > 4:
        height = 500
    else:
        height = 300

    # use mols2grid to display our dataframe with molecules
    raw_html = mols2grid.display(most_similar_df, subset = ["Name", "similarity"],
                                smiles_col = "Smiles")._repr_html_()
    components.html(raw_html, height = height)
else:
    st.error('INVALID MOLECULE', icon="🚨")
```

### Interactive plots
Finally, we will do some graphing! Part of streamlit's appeal is that it lets you create beautiful, interactive plots easily. There are many interactive plotting libraries, but now we'll use pyplot. Create a distribution plot for each of the Lipinski descriptors. Add a vertical line so we can see how our molecule stacks up. The skeleton for this code is already in the file, finish it for the last descriptors. 


### All Done!
Our app is now done! Now we have a beautiful, easy-to-use interface. And it wasn't even too hard to make. 